In [27]:
import pandas as pd
import sqlite3
import struct


# Get Tables from DB into DataFrame

In [28]:
conn = sqlite3.connect("book-rec-sys/books_recommendation.db")
cursor = conn.cursor()

In [5]:

# Query to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print table names
for table in tables:
    print(table[0])

books
user_interactions
sqlite_sequence


In [22]:
def decode_sqlite_binary_result(cursor, rows):
    """
    Converts binary fields in SQLite rows to readable integers and returns column names.
    
    Args:
        cursor (sqlite3.Cursor): Cursor used for the query (for getting column names).
        rows (list of tuples): Rows returned from the query.

    Returns:
        tuple: (column_names, decoded_rows)
    """
    column_names = [desc[0] for desc in cursor.description]
    decoded_rows = []

    for row in rows:
        new_row = []
        for value in row:
            if isinstance(value, bytes) and len(value) == 8:
                try:
                    value = struct.unpack('<Q', value)[0]
                except struct.error:
                    pass  # Leave unchanged if unpacking fails
            new_row.append(value)
        decoded_rows.append(tuple(new_row))

    return column_names, decoded_rows


In [23]:
cursor.execute("SELECT * FROM user_interactions;")
rows = cursor.fetchall()

In [24]:
column_names, decoded_rows = decode_sqlite_binary_result(cursor, rows)

In [25]:
interactions_df = pd.DataFrame(decoded_rows, columns=column_names)

In [26]:
print(interactions_df.head)

<bound method NDFrame.head of       interaction_id  user_id   book_id  is_read  rating  is_reviewed
0                  1        0    452196        1       4            0
1                  2        0     12067        0       0            0
2                  3        0    108905        1       4            1
3                  4        0    178186        1       5            1
4                  5        0    150731        0       0            0
...              ...      ...       ...      ...     ...          ...
1696            1697      226  25876779        1       3            0
1697            1698      226   6369113        0       0            0
1698            1699      226   8449713        0       0            0
1699            1700      227   6320534        1       4            0
1700            1701      227    349473        0       0            0

[1701 rows x 6 columns]>


In [29]:
conn.close()

# EDA